# Demonstration of Well Class

Notebook to demonstrate using the Well Class to 
estimate drawdown or stream depletion using the 
solutions available in the python module.

In [ ]:
import sys
sys.path.insert(1, '../../')
import pycap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Example Depletion
The Well class will estimate depletions for streams.  The stream distances and
streambed conductances, if needed, are passed through dictionaries keyed by 
a stream name or ID.  In this notebook a table of values will be made manually from 
Table 2 from Reeves, H.W., Hamilton, D.A., Seelbach, P.W., and Asher, A.J., 2009, Ground-water-withdrawal component of the Michigan water-withdrawal screening tool: U.S. Geological Survey Scientific Investigations Report 2009–5003, 36 p.
[https://pubs.usgs.gov/sir/2009/5003/]

In [ ]:
stream_table = pd.DataFrame(({'id': 8, 'distance': 14802},
                            {'id': 9, 'distance': 12609.2},
                            {'id': 11, 'distance': 15750.5},
                            {'id': 27, 'distance': 22567.6},
                            {'id': 9741, 'distance': 27565.2},
                            {'id': 10532, 'distance': 33059.5},
                            {'id': 11967, 'distance': 14846.3},
                            {'id': 12515, 'distance': 17042.55},
                            {'id': 12573, 'distance': 11959.5},
                            {'id': 12941, 'distance': 19070.8},
                            {'id': 13925, 'distance': 10028.9}))



## Compute apportionment

The example in the report uses inverse-distance weighting apportionment.  Other
apportionment approaches that may be used as a simple way to extend the analytical
solution are discussed by Zipper and others (2019).  [https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2018WR024403]

In [ ]:
invers =np.array([1/x for x in stream_table['distance']])
stream_table['apportionment'] = (1./stream_table['distance'])/np.sum(invers)

## set aquifer properties and streambed conductances

In [ ]:
T= 7211.  # ft^2/day
S= 0.01
Q = 70  # 70 gpm in cubic feet per day
well_name = 'demo'
pumpdays = int(5. * 365)
stream_table['conductance'] = 7.11855  # example uses constant streambed_conductance

## call a utility to convert Q to a series with appropriate formatting

The Well Class works with a Pandas series of pumping rates, it may be
easily constructed with the Q2ts function.  The index of the
series is pumping days.  We also need to convert 
from pumping in GPM to cubic feet per day so that units are consistent.


In [ ]:
Q = pycap.Q2ts(pumpdays, 5, Q) * pycap.GPM2CFD

In [ ]:
Q

In [ ]:
stream_table.head()

make dictionaries from the distances, apportionment, and conductance values

In [ ]:
distances = dict(zip(stream_table.id.values, stream_table.distance.values))

In [ ]:
apportion = dict(zip(stream_table.id.values, stream_table.apportionment.values))

In [ ]:
cond = dict(zip(stream_table.id.values, stream_table.conductance.values))

## Make a Well object

Depletion and drawdown can be easily returned as attributes of the object.
The choice of depletion or drawdown method is made through a parameter passed to
the object.  In this example the Hunt (1999) depletion method is used
to compute stream depletion for the dictionary of streams at the given
distances with the inverse-distance apportionment.  We also can
compute drawdown at a dictionary of locations.  If drawdown_dist
is not passed to the object, then drawdowns will not be computed.
Results are returned as arrays that may easily be converted to 
Pandas dataframes for ease of viewing and plotting.

In [ ]:
test_well = pycap.Well(T=T,
                 S=S,
                 Q=Q,
                 depletion_years=5,
                 depl_method='hunt99',
                 drawdown_dist={'testlocation0':50.},  #default method is Theis
                 streambed_conductance=cond,
                 stream_dist=distances,
                 stream_apportionment=apportion)

In [ ]:
drawdown = pd.DataFrame(test_well.drawdown)
drawdown.head()

In [ ]:
stream_depl = pd.DataFrame(test_well.depletion)

## need to convert to GPM to compare results to Table 2

The object get daily time series according to the 
pumping schedule.  We can pull out the 5-year depletion
to note that the final results match Table 2.

In [ ]:
stream_depl = stream_depl/pycap.GPM2CFD
five_year= pd.DataFrame(stream_depl.loc[1824].T)
five_year.rename(columns={1824: 'Depletion'}, inplace=True)
five_year

## What about plotting?

In [ ]:
fig, ax = plt.subplots()
drawdown.plot(ax=ax)
ax.set_xlabel('pumping in days')
ax.set_ylabel('drawdown in feet')

In [ ]:
fig, ax = plt.subplots()
stream_depl.plot(ax=ax)
ax.set_xlabel('pumping in days')
ax.set_ylabel('depletion in GPM')
ax.legend(title='Explanation', 
          bbox_to_anchor=(0, 0.9, 1, 0.9), 
          bbox_transform=fig.transFigure, 
          ncols=5,
          mode='expand',
          loc='lower left')


## Intermittent Pumping